<a href="https://colab.research.google.com/github/mohammad-rahbari/federated-learning_visual-classification/blob/main/notebooks/Federated_learning_server_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and data

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from torchvision.datasets import CIFAR100
from torch.utils.data import Subset
import numpy as np
import copy
import random
import torch
import pandas as pd

In [3]:
#@title In this block we import the test set of CIFAR100 to evaluate the global model
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 169M/169M [00:03<00:00, 42.6MB/s]


# Collecting data of models we want to aggregate

In [28]:
initial_model_name = "ed199b6b-9a9e-4465-9ad0-5e8a64c7007f" #@param{"type":"string"}


In [27]:
#@title Client's log file has been loaded in this block so we can use it in next steps
clients_data = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")
clients_data.head()

,client_id,backbone,model_name,initial_model_name,path,num_of_clients,Measurement_criteria,accuracy,loss,train_loss,...,client_train_size,client_test_size,train_test_ratio,classes,round_number,duration,time,path_to_subsets,path_to_class_combs,has_model_edited
0,44,dino_vits16,a31ff492-6902-45bb-9834-cb74c2df6c8a,189c599a-33cb-4ae0-8973-2b63468946d2,/content/drive/MyDrive/MLDL_FederatedLearning/...,80,"accuracy,loss,train_loss",18.858561,3.759583,3.845865,...,403,101,"{'train': 0.8, 'test': 0.2}",all,1,16.959211,2025-07-02 18:26:54,/content/drive/MyDrive/MLDL_FederatedLearning/...,,False
1,79,dino_vits16,ee84e118-20af-44ad-b17a-2de98b116875,189c599a-33cb-4ae0-8973-2b63468946d2,/content/drive/MyDrive/MLDL_FederatedLearning/...,80,"accuracy,loss,train_loss",13.065327,3.917798,4.054595,...,199,50,"{'train': 0.8, 'test': 0.2}",all,1,9.063819,2025-07-02 18:27:05,/content/drive/MyDrive/MLDL_FederatedLearning/...,,False
2,53,dino_vits16,bc7905ef-21ff-4904-a954-93193789ac61,189c599a-33cb-4ae0-8973-2b63468946d2,/content/drive/MyDrive/MLDL_FederatedLearning/...,80,"accuracy,loss,train_loss",16.867470,3.903104,3.980483,...,415,104,"{'train': 0.8, 'test': 0.2}",all,1,17.582530,2025-07-02 18:27:24,/content/drive/MyDrive/MLDL_FederatedLearning/...,,False
3,0,dino_vits16,d7064774-7a7e-4ce1-8592-258b50c7d531,189c599a-33cb-4ae0-8973-2b63468946d2,/content/drive/MyDrive/MLDL_FederatedLearning/...,80,"accuracy,loss,train_loss",23.459716,3.748058,3.797915,...,422,106,"{'train': 0.8, 'test': 0.2}",all,1,17.383544,2025-07-02 18:27:43,/content/drive/MyDrive/MLDL_FederatedLearning/...,,False
4,3,dino_vits16,cc78abfe-41b7-471b-ac52-1e82f540db84,189c599a-33cb-4ae0-8973-2b63468946d2,/content/drive/MyDrive/MLDL_FederatedLearning/...,80,"accuracy,loss,train_loss",17.857143,3.808162,3.891812,...,420,105,"{'train': 0.8, 'test': 0.2}",all,1,18.352425,2025-07-02 18:28:04,/content/drive/MyDrive/MLDL_FederatedLearning/...,,False


In [29]:
filter =  clients_data['initial_model_name']== initial_model_name
filtered_clients_data = clients_data[filter] # Using filter to collect clients with specified initial model
params = filtered_clients_data[['backbone',
                                    'num_of_clients',
                                    'splitting_method',
                                    'size_of_dataset']]
params = dict(params.iloc[0])
print("Number of all trained clients:", len(clients_data))
print("Number of clients after filtering:", len(filtered_clients_data))
contributors = [] # contributors is being used to store the name of models which contributes in aggregation
for i  in filtered_clients_data['model_name'].values:
  contributors.append(i)
print("Contributors:", contributors)
filtered_clients_data.head()

Number of all trained clients: 24
Number of clients after filtering: 12
Contributors: ['d0a29b7b-4da6-4c09-bf5e-e5055fa10c87', '81d37f1f-83d6-41d4-b3fa-b131551ad71e', 'd12e967b-7ce5-4a50-92a3-da22b518fa92', 'a8260b7f-685d-4041-9f43-e1f7821bf580', '5f1a7fec-6b43-40ac-85ed-86ad3d04ae79', '49343ace-81f0-4d21-907a-4663a60fee7b', '2b58dbc0-536f-4c2b-bdcb-041b657cb388', 'd6ab94e9-72f1-4f9f-8945-c613a180f931', '5ccaf6ea-edd4-46ce-9c34-7f367c7298c9', '7e77fdc3-b585-4225-9f21-c4f7719fce49', '5e8f2190-7680-42c5-85b7-d62afd7667e8', '256b3daf-227e-4525-85d0-2d14b6ad4b7a']


,client_id,backbone,model_name,initial_model_name,path,num_of_clients,Measurement_criteria,accuracy,loss,train_loss,...,client_train_size,client_test_size,train_test_ratio,classes,round_number,duration,time,path_to_subsets,path_to_class_combs,has_model_edited
12,22,dino_vits16,d0a29b7b-4da6-4c09-bf5e-e5055fa10c87,ed199b6b-9a9e-4465-9ad0-5e8a64c7007f,/content/drive/MyDrive/MLDL_FederatedLearning/...,80,"accuracy,loss,train_loss",23.341523,3.566201,3.660084,...,407,102,"{'train': 0.8, 'test': 0.2}",all,2,19.611794,2025-07-03 13:18:52,/content/drive/MyDrive/MLDL_FederatedLearning/...,,False
13,67,dino_vits16,81d37f1f-83d6-41d4-b3fa-b131551ad71e,ed199b6b-9a9e-4465-9ad0-5e8a64c7007f,/content/drive/MyDrive/MLDL_FederatedLearning/...,80,"accuracy,loss,train_loss",28.467153,3.358154,3.431519,...,411,103,"{'train': 0.8, 'test': 0.2}",all,2,19.841092,2025-07-03 13:19:14,/content/drive/MyDrive/MLDL_FederatedLearning/...,,False
14,76,dino_vits16,d12e967b-7ce5-4a50-92a3-da22b518fa92,ed199b6b-9a9e-4465-9ad0-5e8a64c7007f,/content/drive/MyDrive/MLDL_FederatedLearning/...,80,"accuracy,loss,train_loss",25.290698,3.450663,3.504752,...,344,86,"{'train': 0.8, 'test': 0.2}",all,2,17.167468,2025-07-03 13:19:33,/content/drive/MyDrive/MLDL_FederatedLearning/...,,False
15,14,dino_vits16,a8260b7f-685d-4041-9f43-e1f7821bf580,ed199b6b-9a9e-4465-9ad0-5e8a64c7007f,/content/drive/MyDrive/MLDL_FederatedLearning/...,80,"accuracy,loss,train_loss",29.166667,3.438611,3.523140,...,408,102,"{'train': 0.8, 'test': 0.2}",all,2,19.199889,2025-07-03 13:19:55,/content/drive/MyDrive/MLDL_FederatedLearning/...,,False
16,26,dino_vits16,5f1a7fec-6b43-40ac-85ed-86ad3d04ae79,ed199b6b-9a9e-4465-9ad0-5e8a64c7007f,/content/drive/MyDrive/MLDL_FederatedLearning/...,80,"accuracy,loss,train_loss",25.129534,3.536212,3.544303,...,386,97,"{'train': 0.8, 'test': 0.2}",all,2,18.727870,2025-07-03 13:20:15,/content/drive/MyDrive/MLDL_FederatedLearning/...,,False


In [21]:
#@title Dino Model
import torch
import torch.nn as nn

class DinoClassifier(nn.Module):
  def __init__(self, backbone, num_classes:int=100, device=None):
    super(DinoClassifier, self).__init__()
    self.backbone = torch.hub.load('facebookresearch/dino:main', backbone)


    #We need to freaze thhe parameters of bakbone first so we can train only on the head layer(output layer)
    for param in self.backbone.parameters():
      param.requires_grad = False

    #determine the Device
    if device is None:
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    self.backbone.to(device)

    #To detect the output feature dimontion of backbone we run  Dummy forward pass
    with torch.no_grad():

      dummy_input = torch.randn(1,3,224,224).to(device)
      dummy_out = self.backbone(dummy_input)


      if isinstance(dummy_out, tuple):
        dummy_out = dummy_out[0]
      elif isinstance(dummy_out, dict):
        dummy_out = dummy_out.get("x_norm_clstoken", next(iter(dummy_out.values())))

      #If the output is 3D (B, T, D), we assume first token is the [CLS] token.
      if dummy_out.dim() == 3:
        dummy_feature = dummy_out[:,0]
      else:
        dummy_feature = dummy_out
      feature_dim = dummy_feature.shape[1]
      print("Detected feature dimontion:", feature_dim)


      #Hidden Layer
      self.hidden = nn.Sequential(
          nn.Linear(feature_dim, 128),
          nn.ReLU()
      )


      #Difineing the classification Head
      self.head = nn.Linear(128, num_classes)

      #Ensure the head is trainable.
      for param in self.hidden.parameters():
        param.requires_grad = True
      for param in self.head.parameters():
        param.requires_grad = True

  def forward(self,x):

    #pass the input through the backbone
    features = self.backbone(x)

    if isinstance(features, tuple):
      features = features[0]
    elif isinstance(features, dict):
      features = features.get("x_norm_clstoken", next(iter(features.values())))


    # If featers are retuened as (B, T, D), use the first token
    if features.dim() == 3:
      features = features[:,0]


    hidden_out  = self.hidden(features)

    logits = self.head(hidden_out)

    return logits


In [9]:
# @title `get_model` function retrieves and loads the models of filtered clients
def get_model(paths,sample_sizes, backbone):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = DinoClassifier(backbone=backbone, num_classes=100, device=device) # Loading an initial custom dino model
  for index in range(len(paths)):
    state_dict = torch.load(paths.iloc[index]) # load state dict regarding the client number 'index'
    model.head.load_state_dict(state_dict["head"]) # set the state dict based on client
    model.hidden.load_state_dict(state_dict["hidden"]) # set the state dict based on client

    model.to(device)
    yield (model,sample_sizes.iloc[index]) # this command throws model one at the time so less time and resouces will be used

# Aggregation functions

Implemented algorithm:

*   FebAvg
*   FebAvgM
*   EMA



In [10]:
#@title <h2>FebAvg</h2>
def feb_avg(df):
  total_samples = df["client_train_size"].sum() # Calculate the total number of samples of clients wich had contributed
  global_weights = {"head":None, "hidden":None} # This variable stores the weights we want to modify

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if global_weights["head"] is None:
        global_weights["head"] = copy.deepcopy(model.head.state_dict())
        global_weights["hidden"] = copy.deepcopy(model.hidden.state_dict())
        global_model = copy.deepcopy(model)
        for k in global_weights["head"].keys():
          global_weights["head"][k].zero_() # This command sets the tensor to zero
        for k in global_weights["hidden"].keys():
          global_weights["hidden"][k].zero_() # This command sets the tensor to zero

      for k in global_weights["head"].keys():
        global_weights["head"][k] += model.head.state_dict()[k] * (sample_size / total_samples) # Each weight will be assgin by average of all clients weights
      for k in global_weights["hidden"].keys():
        global_weights["hidden"][k] += model.hidden.state_dict()[k] * (sample_size / total_samples) # Each weight will be assgin by average of all clients weights
    global_model.head.load_state_dict(global_weights["head"]) # A model with modified head will be assignd
    global_model.hidden.load_state_dict(global_weights["hidden"]) # A model with modified hidden will be assignd
  return global_model



In [11]:
#@title FebAvgM
def feb_avg_m(df, momentum_coefficient=0.9, momentum_vector_path= None):

  total_samples = df["client_train_size"].sum() # Calculate the total number of samples of clients wich had contributed
  delta = {"head":None, "hidden":None} # `delta` is variable that keep the average of clients
  global_weights = {"head":None, "hidden":None} # This variable stores the weights we want to modify

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if delta['head'] is None:

        global_model = copy.deepcopy(model)
        global_weights["head"] = copy.deepcopy(model.head.state_dict())
        global_weights["hidden"] = copy.deepcopy(model.hidden.state_dict())
        delta["head"] = { k: torch.zeros_like(v) for k, v in global_weights["head"].items() } # A dict with structure of the model that we want to modify will be generated
        delta["hidden"] = { k: torch.zeros_like(v) for k, v in global_weights["hidden"].items() } # A dict with structure of the model that we want to modify will be generated


      client_head = model.head.state_dict()
      for k in delta['head'].keys():
        delta['head'][k] += (client_head[k] - global_weights["head"][k]) * (sample_size / total_samples) # Each weight will be assgin by average of all clients weights

      client_hidden = model.hidden.state_dict()
      for k in delta['hidden'].keys():
        delta['hidden'][k] += (client_hidden[k] - global_weights["hidden"][k]) * (sample_size / total_samples) # Each weight

    # In this section we calculate the momentum_vector


    if momentum_vector_path is None:
      momentum_vector = {
          "head": {k: delta["head"][k].clone() for k in delta["head"]},
          "hidden": {k: delta["hidden"][k].clone() for k in delta["hidden"]}
      }
    else:
      momentum_vector = torch.load(momentum_vector_path) #In rounds > 1 momentum vector is requerd to be loaded from drive
      for k in delta['head'].keys():
        momentum_vector['head'][k] = momentum_coefficient * momentum_vector['head'][k]  + delta['head'][k]  # Using the the formula of FebAvgM we calculate the momentum vector
      for k in delta['hidden'].keys():
        momentum_vector['hidden'][k] = momentum_coefficient * momentum_vector['hidden'][k]  + delta['hidden'][k]  # Using the the formula of FebAvgM we calculate the momentum vector






    global_head = copy.deepcopy(global_model.head.state_dict())
    for k in momentum_vector['head'].keys():
      global_head[k] =global_head[k] + momentum_vector['head'][k] #After adding momentum vector the last global model we use clamp function we insure to keep momentum vector in boundary
    global_hidden = copy.deepcopy(global_model.hidden.state_dict())
    for k in momentum_vector['hidden'].keys():
      global_hidden[k] = global_hidden[k] + momentum_vector['hidden'][k]#After adding momentum vector the last global model we use clamp function


    global_model.head.load_state_dict(global_head)
    global_model.hidden.load_state_dict(global_hidden)


  return global_model, momentum_vector # We return momentum_vector to save it and use for next aggregation steps



In [12]:
#@title EMA
def EMA(df, decay=0.9, momentum_vector_path= None):

  total_samples = df["client_train_size"].sum() # Calculate the total number of samples of clients wich had contributed
  delta = {"head":None, "hidden":None} # `delta` is variable that keep the average of clients
  global_weights = {"head":None, "hidden":None} # This variable stores the weights we want to modify

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"])
  with torch.no_grad():

    for model, sample_size in models:
      if delta['head'] is None:

        global_model = copy.deepcopy(model)
        global_weights["head"] = copy.deepcopy(model.head.state_dict())
        global_weights["hidden"] = copy.deepcopy(model.hidden.state_dict())
        delta["head"] = { k: torch.zeros_like(v) for k, v in global_weights["head"].items() } # A dict with structure of the model that we want to modify will be generated
        delta["hidden"] = { k: torch.zeros_like(v) for k, v in global_weights["hidden"].items() } # A dict with structure of the model that we want to modify will be generated


      client_head = model.head.state_dict()
      for k in delta['head'].keys():
        delta['head'][k] += (client_head[k] - global_weights["head"][k]) * (sample_size / total_samples) # Each weight will be assgin by average of all clients weights

      client_hidden = model.hidden.state_dict()
      for k in delta['hidden'].keys():
        delta['hidden'][k] += (client_hidden[k] - global_weights["hidden"][k]) * (sample_size / total_samples) # Each weight

    # In this section we calculate the momentum_vector


    if momentum_vector_path is None:
      momentum_vector = {
          "head": {k: delta["head"][k].clone() for k in delta["head"]},
          "hidden": {k: delta["hidden"][k].clone() for k in delta["hidden"]}
      }
    else:
      momentum_vector = torch.load(momentum_vector_path) #In rounds > 1 momentum vector is requerd to be loaded from drive
      for k in delta['head'].keys():

        momentum_vector['head'][k] = decay * momentum_vector['head'][k]  + (1- decay) *  delta['head'][k]  # Using the the formula of EMA we calculate the momentum vector
      for k in delta['hidden'].keys():
        momentum_vector['hidden'][k] = decay * momentum_vector['hidden'][k]  + (1- decay) *  delta['hidden'][k]  # Using the the formula of FebAvgM we calculate the momentum vector






    global_head = copy.deepcopy(global_model.head.state_dict())
    for k in momentum_vector['head'].keys():
      global_head[k] =global_head[k] + momentum_vector['head'][k] #After adding momentum vector the last global model we use clamp function we insure to keep momentum vector in boundary
    global_hidden = copy.deepcopy(global_model.hidden.state_dict())
    for k in momentum_vector['hidden'].keys():
      global_hidden[k] = global_hidden[k] + momentum_vector['hidden'][k]#After adding momentum vector the last global model we use clamp function


    global_model.head.load_state_dict(global_head)
    global_model.hidden.load_state_dict(global_hidden)


  return global_model, momentum_vector # We return momentum_vector to save it and use for next aggregation steps




In [13]:
# @title This function will evaluate the model.</br> The outputs are loss and accuracy
def evaluation(model, data_loader):
  criterion = nn.CrossEntropyLoss()
  model.eval()
  correct = 0
  total = 0
  test_loss = 0

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  with torch.no_grad():
    for images, labels in  data_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)

      _, prediction = torch.max(outputs.data,1)
      loss = criterion(outputs, labels)
      test_loss += loss.item() * labels.size(0)

      total += labels.size(0)
      correct += (prediction == labels).sum().item()
    accuracy = 100 * correct / total
    loss = test_loss / total
    return accuracy, loss



In [14]:
# @title <h2>`next_id`</h2> This function generates a unique name for model. `uuid4` does not generate duplicated but we are using a fixed `seed` hence we insure this name does not already exists.

from uuid import uuid4
import os
def next_id(log_path):
  if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    while True:
      uuid = str(uuid4())
      if uuid not in df["model_name"].values:
        return uuid
  else:
    return str(uuid4())

In [15]:
from datetime import datetime

def get_current_time():
  now = datetime.now()

  formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S") # Format the date and time as a string

  return formatted_date_time

def global_model_name_path_generator():

  model_name = next_id("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")

  path = "/content/drive/MyDrive/MLDL_FederatedLearning/models/global/" + model_name + ".pth"

  return model_name, path



In [16]:
def del_model(model_name):
  log_df = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
  filter = log_df["model_name"] == model_name
  if not filter.any():
    print(f"recored ({model_name}) not found.")
    return
  if os.path.exists(log_df[filter]["path"].values[0]):
    os.remove(log_df[filter]["path"].values[0])
  else:
    print("model not found")


  if not np.isnan(log_df[filter]["momentum_vector_path"].values[0])  and os.path.exists(log_df[filter]["momentum_vector_path"].values[0]):
    os.remove(log_df[filter]["momentum_vector_path"].values[0])
  else:
    print("momentum vector not found")
  log_df = log_df[~filter]
  log_df.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv", index=False)
# del_model("54ba3a96-df4d-4e55-bea1-fa8547aff906")

In [ ]:
# #@title use this block to modify the global log file
# temp = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
# temp["aggregation_method"] =[ np.nan] * len(temp)
# temp["contributors"] =[ np.nan] * len(temp)
# temp["momentum_vector_path"] = [np.nan] * len(temp)

# temp = temp[['backbone',
#               'num_of_clients',
#               'splitting_method',
#               'aggregation_method',
#               'Measurement_criteria',
#               'accuracy',
#               'loss',
#               'size_of_dataset',
#               'train_test_ratio',
#               'classes',
#               'round_number',
#               'num_of_participants',
#               'model_name',
#               'prev_global_model_name',
#               "contributors",
#               'path',
#               "momentum_vector_path",
#               'path_to_subsets',
#               'path_to_class_combs',
#               'time'
#                ]]
# temp.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv", index=False)
# temp.head()
# del temp

In [37]:
#@title <h1>Select aggregation method </h1>
#@markdown This value will be auto assigned in case the initial model of filtered clients have been aggregated with a spicific aggregation function in the previous rounds

aggregation_method = "EMA"   #@param["FebAvg","FebAvgM", "EMA" ]

prev_agg_method = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")

filter = prev_agg_method["model_name"] == initial_model_name
prev_agg_method = prev_agg_method[filter]

prev_agg_method = prev_agg_method["aggregation_method"].values[0]

aggregation_method = aggregation_method if not type(prev_agg_method) == type("str") else prev_agg_method


print(f"{aggregation_method} has been selected as the Aggregation function.")

EMA has been selected as the Aggregation function.


In [38]:
if aggregation_method == "FebAvg":
  global_model = feb_avg(filtered_clients_data)
elif aggregation_method == "FebAvgM":
  global_model,momentum_vector = feb_avg_m(filtered_clients_data)
elif aggregation_method == "EMA":
  global_model,momentum_vector = EMA(filtered_clients_data)



model_name, path = global_model_name_path_generator()

test_accracy, test_loss= evaluation(global_model, test_loader)
print(f"Accurace:{test_accracy:.2f}")
print(f"Loss:{test_loss:.2f}")

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


Detected feature dimontion: 384
Accurace:11.88
Loss:4.05


In [39]:
import os
log_path = "/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv"

prev_global_model_name = filtered_clients_data["initial_model_name"].values[0]
global_model_log = filtered_clients_data.drop(["client_id","train_loss","client_train_size","client_test_size","duration",],axis=1)
global_model_log = global_model_log.iloc[0]
global_model_log["num_of_participants"] = len(filtered_clients_data)
global_model_log["prev_global_model_name"] = initial_model_name
global_model_log["model_name"]= model_name
global_model_log["accuracy"] = test_accracy
global_model_log["loss"] = test_loss
global_model_log["time"] = get_current_time()
global_model_log["path"] = path
global_model_log["Measurement_criteria"] = "accuracy,loss"
global_model_log["contributors"] = contributors
global_model_log["aggregation_method"] = aggregation_method

if aggregation_method == "FebAvg":
  global_model_log["momentum_vector_path"] = None

elif aggregation_method == "FebAvgM" or aggregation_method == "EMA":
  global_model_log["momentum_vector_path"] = "/content/drive/MyDrive/MLDL_FederatedLearning/models/global/momentun_vectors/MV_"+ model_name + ".pt"
  torch.save(momentum_vector, global_model_log["momentum_vector_path"])


global_model_log = pd.DataFrame(global_model_log).T
global_model_log = global_model_log[['backbone',
              'num_of_clients',
              'splitting_method',
              'aggregation_method',
              'Measurement_criteria',
              'accuracy',
              'loss',
              'size_of_dataset',
              'train_test_ratio',
              'classes',
              'round_number',
              'num_of_participants',
              'model_name',
              'prev_global_model_name',
              "contributors",
              'path',
              "momentum_vector_path",
              'path_to_subsets',
              'path_to_class_combs',
              'time'
               ]]

flag = False
if os.path.exists(path):
  print("Model already exist")
  flag = True

if os.path.exists(log_path):
  global_log_df = pd.read_csv(log_path)
  if model_name in global_log_df["model_name"].values:
    print("record already exist")
    flag = True
  if not flag:
    global_model_log.to_csv(log_path, mode='a', header=False, index=False)
else:
  if not flag:
    global_model_log.to_csv(log_path, index=False, header=True)


if not flag:
  torch.save(global_model.state_dict(),  path )
  if aggregation_method == "FebAvgM" or aggregation_method == "EMA":
    torch.save(momentum_vector, global_model_log["momentum_vector_path"].values[0])


In [40]:
global_model_log.head()

,backbone,num_of_clients,splitting_method,aggregation_method,Measurement_criteria,accuracy,loss,size_of_dataset,train_test_ratio,classes,round_number,num_of_participants,model_name,prev_global_model_name,contributors,path,momentum_vector_path,path_to_subsets,path_to_class_combs,time
12,dino_vits16,80,non-i.i.d. sharing,EMA,"accuracy,loss",11.88,4.048569,50000,"{'train': 0.8, 'test': 0.2}",all,2,12,888244c0-ad51-42ff-adc6-38c8179c6ad5,ed199b6b-9a9e-4465-9ad0-5e8a64c7007f,"[d0a29b7b-4da6-4c09-bf5e-e5055fa10c87, 81d37f1...",/content/drive/MyDrive/MLDL_FederatedLearning/...,/content/drive/MyDrive/MLDL_FederatedLearning/...,/content/drive/MyDrive/MLDL_FederatedLearning/...,,2025-07-03 13:30:46
